In [1]:
import os
os.chdir('../')
%pwd

'/home/ehsan/PycharmProjects/Chicken-Disease-Classification/Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path 
    trained_model_path: Path
    updated_base_model_path: Path
    training_data_path: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmented: bool
    params_image_size: list
    

In [3]:
from cnnClassifier.entity.config_entity import CallbacksConfig

In [4]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_dirs
import tensorflow as tf

2024-02-22 12:38:30.603580: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-02-22 12:38:35,521 - INFO - utils - NumExpr defaulting to 8 threads.


In [13]:
class ConfigurationManager:
    def __init__(
            self,
            config_path = CONFIG_FILE_PATH,
            params_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_path)
        
        create_dirs([self.config.artifacts_root])
        
    
    def get_callbacks_config(self) -> CallbacksConfig:
        config1 = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config1.checkpoint_model_file)
        create_dirs([
            model_ckpt_dir,
            config1.tensorboard_log_dir
                     ])
        callbacks_config1 = CallbacksConfig(
            root_dir=Path(config1.root_dir),
            tensorboard_log_dir=Path(config1.tensorboard_log_dir),
            checkpoint_dir=Path(config1.checkpoint_model_file)
        )
        return callbacks_config1
   
    def get_training_config(self) -> TrainingConfig:
       training_config = self.config.training
       prepare_base_model = self.config.prepare_base_model
       params = self.params
       training_data = os.path.join(self.config.data_ingestion.unzip_file, "Chicken-fecal-images")
       create_dirs([training_config.root_dir])
       
       training_config = TrainingConfig(
           root_dir=training_config.root_dir,
           trained_model_path=Path(training_config.trained_model_path),
           updated_base_model_path =Path(prepare_base_model.updated_base_model_path),
           training_data_path=training_data,
           params_epochs=params.EPOCHS,
           params_batch_size=params.BATCH_SIZE,
           params_is_augmented=params.AUGMENTATION,
           params_image_size=params.IMAGE_SIZE
       )
       return training_config

In [9]:
import time

In [10]:
class Callbacks:
    def __init__(self, config: CallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d--%H:%M:%S")
        tb_running_log_dir = os.path.join(self.config.tensorboard_log_dir, f'tb_log_{timestamp}')
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_dir,
            save_best_only=True
        )

    def get_tb_ckpt_callbacks(self):
        return [self._create_tb_callbacks,
                self._create_ckpt_callbacks]
    

In [15]:
class Trainer:
    def __init__(self, config: TrainingConfig):
        self.validation_steps = None
        self.step_per_epochs = None
        self.train_generator = None
        self.valid_data_generator = None
        self.model = None
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)
        
    def train_valid_generator(self):
        data_generator_kwargs = dict(
            rescale=1./255,
            validation_split=0.2
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation='bilinear'
        )
        valid_data_generator = tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs
            )
        self.valid_data_generator = valid_data_generator.flow_from_directory(
            directory=self.config.training_data_path,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )
        if self.config.params_is_augmented:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **data_generator_kwargs
            )
        else:
            train_datagen = valid_data_generator
            
        self.train_generator = train_datagen.flow_from_directory(
            directory=self.config.training_data_path,
            subset='training',
            shuffle=True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def train(self, callbacks_list: list):
        self.step_per_epochs = self.train_generator.samples // self.train_generator.batch_size
        self.validation_steps = self.valid_data_generator.samples // self.valid_data_generator.batch_size
        
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            callbacks=callbacks_list,
            steps_per_epoch=self.step_per_epochs,
            validation_steps=self.validation_steps,
            validation_data=self.valid_data_generator
        )
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )

In [16]:
try:
    config = ConfigurationManager()
    callbacks_config = config.get_callbacks_config()
    callbacks_list = Callbacks(callbacks_config)
    callbacks_list = callbacks_list.get_tb_ckpt_callbacks()
    
    training_config = config.get_training_config()
    trainer = Trainer(training_config)
    trainer.get_base_model()
    trainer.train_valid_generator()
    trainer.train(callbacks_list)
except Exception as e:
    raise e

2024-02-22 13:16:40,803 - INFO - common - Loaded yaml file config/config.yaml
2024-02-22 13:16:40,811 - INFO - common - Loaded yaml file params.yaml
2024-02-22 13:16:40,814 - INFO - common - created directory artifacts
2024-02-22 13:16:40,816 - INFO - common - created directory artifacts/prepare_callbacks/checkpoint_dir
2024-02-22 13:16:40,819 - INFO - common - created directory artifacts/prepare_callbacks/tensorboard_log_dir
2024-02-22 13:16:40,821 - INFO - common - created directory artifacts/training
Found 78 images belonging to 2 classes.
Found 312 images belonging to 2 classes.
Epoch 1/5


2024-02-22 13:16:41.754987: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


19/19 [==============================] - ETA: 0s - loss: 13.7088 - accuracy: 0.5101

2024-02-22 13:17:49.410050: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


19/19 [==============================] - 109s 6s/step - loss: 13.7088 - accuracy: 0.5101 - val_loss: 15.6933 - val_accuracy: 0.3906
Epoch 2/5
19/19 [==============================] - 79s 4s/step - loss: 9.1934 - accuracy: 0.5878 - val_loss: 3.4977 - val_accuracy: 0.6875
Epoch 3/5
19/19 [==============================] - 60s 3s/step - loss: 2.8906 - accuracy: 0.7703 - val_loss: 0.9250 - val_accuracy: 0.9062
Epoch 4/5
19/19 [==============================] - 57s 3s/step - loss: 2.4565 - accuracy: 0.8243 - val_loss: 2.6250 - val_accuracy: 0.7188
Epoch 5/5
19/19 [==============================] - 75s 4s/step - loss: 3.2076 - accuracy: 0.7703 - val_loss: 6.5765 - val_accuracy: 0.5625
